In [ ]:
import pickle
import torch
import os

pretrain_model_path = '/data/LPJ/ICML25/all_checkpoints/pretrain_gnn_qformer_havenllama_using_1989_57_without_lora/v0_50epoch_separate_lr_gnn1e3_qformer_5e4/haven_llama_qformer_1989_57_pretrain_without_lora_50epoch_separate_lr_gnn1e3_qformer_5e4.ckpt'
pretrain_model = torch.load(pretrain_model_path, map_location='cpu')
new_gnn_path = '/data/LPJ/ICML25/all_checkpoints/graph_tower/pretrain_gnn_qformer_havenllama_using_1989_57_without_lora/v0_50epoch_separate_lr_gnn1e3_qformer_5e4'
new_gnn_path = os.path.join(new_gnn_path, 'clip_gt_arxiv')
os.makedirs(new_gnn_path, exist_ok=True)
new_projector_path = '/data/LPJ/ICML25/all_checkpoints/projector/pretrain_gnn_qformer_havenllama_using_1989_57_without_lora/v0_50epoch_separate_lr_gnn1e3_qformer_5e4'
os.makedirs(new_projector_path, exist_ok=True)

new_gnn_weights_path = os.path.join(new_gnn_path, 'model_weights.pkl')

In [4]:
orig_gnn_path = '/data/LPJ/ICML25/GraphGPT/clip_gt_arxiv/clip_gt_arxiv_pub.pkl'
orig_path = '/data/LPJ/ICML25/GraphGPT/clip_gt_arxiv'
orig_gnn = torch.load(orig_gnn_path, map_location='cpu')
##copy config.json from orig_gnn_path to new_gnn_path
import shutil
shutil.copyfile(os.path.join(orig_path, 'config.json'), os.path.join(new_gnn_path, 'config.json'))


'/data/LPJ/ICML25/all_checkpoints/graph_tower/pretrain_gnn_qformer_havenllama_using_1989_57_without_lora/v0_50epoch_separate_lr_gnn1e3_qformer_5e4/clip_gt_arxiv/config.json'

In [5]:
new_graph = {}
for key in pretrain_model['state_dict']:
    if 'graph_tower' in key:
        index = key.find('graph_tower')
        new_key = "gnn" + key[index + len('graph_tower'):]
        new_graph[new_key] = pretrain_model['state_dict'][key]

In [6]:
for key in orig_gnn:
    if key not in new_graph:
        new_graph[key] = orig_gnn[key]

In [7]:
torch.save(new_graph, new_gnn_weights_path)

In [ ]:
# from graphgpt.model.GraphLlama import Qformer_proj
# qfomer = Qformer_proj(graph_hidden_size=128, num_query_token=24, bert_name="/data/LPJ/bert/bert-L12-H128-uncased", cross_attention_freq=2, llm_hidden_size=4096)


/home/yiyao_yang/anaconda3/envs/graphgpt/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
new_projector = {}
for key in pretrain_model['state_dict']:
    if 'graph_projector' in key:
        index = key.find('graph_projector')
        new_key = key[index + len('graph_projector') + 1:]
        new_projector[new_key] = pretrain_model['state_dict'][key]

In [9]:
torch.save(new_projector, os.path.join(new_projector_path, 'projector.bin'))

In [4]:
import torch
new_projector_path = '/data/LPJ/ICML25/all_checkpoints/projector/pretrain_separate_lr_gnn8e3_projector3e4_without_lora/projector.bin'
new_projector = torch.load(new_projector_path, map_location='cpu')

In [5]:
new_projector

OrderedDict([('weight',
              tensor([[ 0.0713, -0.0452, -0.0811,  ...,  0.0364,  0.0145,  0.0156],
                      [-0.0698,  0.0664, -0.0576,  ...,  0.0488,  0.0039, -0.0408],
                      [ 0.0525, -0.0732, -0.0625,  ...,  0.0659, -0.0260,  0.0251],
                      ...,
                      [-0.0752,  0.0537,  0.0253,  ...,  0.0175, -0.0579,  0.0396],
                      [ 0.0767,  0.0312, -0.0542,  ...,  0.0610, -0.0366,  0.0566],
                      [ 0.0452,  0.0815, -0.0205,  ...,  0.0403, -0.0043, -0.0193]])),
             ('bias',
              tensor([-0.0688,  0.0850,  0.0771,  ...,  0.0864,  0.0801, -0.0762]))])

In [6]:
new_projector['model.embed_tokens.weight']

KeyError: 'model.embed_tokens.weight'